In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import cv2

import numpy as np
import mediapipe as mp
import scipy.io

In [ ]:
# Image settings
img_height, img_width = 64, 64
batch_size = 32

# Paths
train_path = '../Datasets/Eye_Direction/TrainingSet/TrainingSet'
test_path = '../Datasets/Eye_Direction/TestSet/TestSet'

# 1. Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Important: set 20% aside for validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

# 2. Create Data Generators

# Training generator (80% of training set)
train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    seed=123
)

# Validation generator (20% of training set)
val_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation',
    seed=123
)

# Testing generator (no shuffle here)
test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# --- NEW PART: Limit steps per epoch ---
small_steps_per_epoch = max(1, train_gen.samples // (10 * batch_size))
small_validation_steps = max(1, val_gen.samples // (10 * batch_size))
